In [3]:
import scipy.io as io
import plotly.offline as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import json
import numpy as np

py.init_notebook_mode(connected=True)

In [4]:
mat = io.loadmat('data_Mg_GBperatom_seg_2Al_dump.mat')

length_A = mat['A'].shape[1]

def draw_peratom_energy(mat , gb_num, export = False):
    i = gb_num
    
    #extract data from the specified Grain Boundary. 
    atom_pos = mat['A']['peratom'][0,i][0,0]['pos']
    
    segE = mat['A']['Eseg'][0,i]
    #check whether this is a valid data?
    n1 = segE[:,0] != 0 
    segE = segE[n1,:]
    
    atom_ID = segE[:,0].astype(int) - 1
    atom_zaxis = atom_pos[atom_ID,2]
    
    gbE = mat['A']['GBE'][0,i][0][0]
    gb_tilt = mat['A']['GB_tilt'][0,i][0]
    gb_norm1 = mat['A']['GB_norm1'][0,i][0]
    
    #draw scatter plot of the data
    main = go.Scatter(x = atom_zaxis-min(atom_zaxis)-20, 
                  y = segE[:,1], 
                  marker={'color':'red', 'symbol':'circle-dot','size':3}, 
                  mode='markers',
                  name='Test'
                 )
    
    data = [main]
    
    layout = go.Layout(autosize = False, 
                   height = 800,
                   width = 800, 
                   xaxis={'title':'Distance from GB', 'zeroline':False},
                   yaxis = {'title':'Segeregation Energy(eV)','zeroline':False},
                   shapes =[{'type':'line', 
                            'x0':-25,
                            'x1':25,
                            'y0':np.mean(segE[:,1]),
                            'y1':np.mean(segE[:,1]),
                            'line':{'width':0.7
                                
                            }
                            }],
                   annotations = [
                       dict(
                       x = 0.08,
                       y = 0.925,
                       xref = 'paper',
                       yref = 'paper',
                       text = '$\\text{GB Energy: %4.1f}\ mJ/m^2$'%(gbE),
                       showarrow = False,
                       font = dict(size = 10)
                       ),
                       dict(
                       x = 0.08,
                       y = 0.90,
                       xref = 'paper',
                       yref = 'paper',
                       text = '$\\text{GB Tilt: [%d %d %d %d]}$'%(gb_tilt[0],
                                                              gb_tilt[1],
                                                              gb_tilt[2],
                                                              gb_tilt[3]),
                       showarrow = False,
                       font = dict(size = 10)
                       ),
                       dict(
                       x = 0.08,
                       y = 0.875,
                       xref = 'paper',
                       yref = 'paper',
                       text = '$\\text{GB Normal: [%d %d %d %d]}$'%(gb_norm1[0],
                                                                   gb_norm1[1],
                                                                   gb_norm1[2],
                                                                   gb_norm1[3]
                       ),
                       showarrow = False,
                       font = dict(size = 10)
                       ),
                       dict(
                       x = 0.08,
                       y = 0.08,
                       xref = 'paper',
                       yref = 'paper',
                       text = '$\\text{Min Segregation: %4.2f eV}$'%(min(segE[:,1])),
                       showarrow = False,
                       font = dict(size = 10)
                       ),
                   ]
                  )
    plot = py.iplot(go.Figure(data = data, layout = layout))

In [19]:
draw_peratom_energy(mat , 15)

In [20]:
def draw_pos_segE(mat, atom_num, export = False):
    i = atom_num
    atom_pos = mat['A']['peratom'][0,i][0,0]['pos']
    num_atom = mat['A']['natoms'][0,i][0][0]
    segE = mat['A']['Eseg'][0,i]
    #check whether this is a valid data?
    n1 = segE[:,1] != 0 
    segE = segE[n1,:]
    atom_ID = segE[:,0].astype(int) - 1
    atom_zaxis = atom_pos[atom_ID,2]
    
    #atomsize_1 = np.ptp(atom_pos[atom_ID,0]) < np.ptp(atom_pos[atom_ID,2])
    #if atomsize_1:
    atomsize = 100*np.ptp(atom_pos[atom_ID,0])*np.ptp(atom_pos[atom_ID,2])/num_atom
    #else:
    #   atomsize = 80*np.around(100-10*np.ptp(atom_pos[atom_ID,0])/np.ptp(atom_pos[atom_ID,2]))/100
    
    main = go.Scatter(x = atom_pos[atom_ID,0], 
              y = atom_pos[atom_ID,2],
              text = np.around(segE[:,1],3).tolist(),
              hoverinfo = 'text',
              marker={'size':atomsize/1.5, 'color':segE[:,1], 'colorscale':'Rainbow' ,'symbol':'circle-dot','showscale':True}, 
              mode='markers'
             )
    data = [main]

    layout = go.Layout(autosize = False,
                   height = 800,
                   width = 1200/np.ptp(atom_pos[atom_ID,2])*np.ptp(atom_pos[atom_ID,0]),     
                   xaxis={'zeroline':False},
                   yaxis = {'zeroline':False}   
                  )
    plot = py.iplot(go.Figure(data = data, layout = layout))

In [21]:
draw_pos_segE(mat, 5)

In [22]:
def draw_correlation_segE_descriptor(mat, descriptor_Name, subnum, export = False):
#################################################################################
#     Input:                                                                    #
#      mat: It is the data source from which we wish to extract the descriptor  #
#           and the segregation energy we want.                                 #
#      descriptor_name: The name of the physical descriptor. Some of the descr  #
#           iptor may not only have one term.                                   #
#      subnum: subnum is the term of the descriptor we wish to plot.            #
#                                                                               #
#     Output:                                                                   #
#      the segeregation energy vs the descriptor plot.                          #
#################################################################################
    for i in range(length_A):
        if i == 0:
            segE = mat['A']['Eseg'][0,0]
            #check whether this is a valid data?
            n1 = segE[:,0] != 0 
            segE = segE[n1,:]

            atom_ID = segE[:,0].astype(int) - 1
            descriptor_temp = mat['A']['peratom'][0,0][0,0][descriptor_Name][:,0]
            descriptor_all = descriptor_temp[atom_ID]
            segE_all = segE
        else:
            segE = mat['A']['Eseg'][0,i]
            #check whether this is a valid data?
            n1 = segE[:,0] != 0 
            segE = np.squeeze(segE[n1,:])

            atom_ID = segE[:,0].astype(int) - 1
            descriptor_temp = mat['A']['peratom'][0,i][0,0][descriptor_Name][:,0]
            descriptor_temp = descriptor_temp[atom_ID]
            descriptor_all = np.concatenate([descriptor_all, descriptor_temp])
            segE_all = np.concatenate([segE_all, segE])
    
    #calculate correlation coefficient
    R = np.corrcoef(np.squeeze(descriptor_all), segE_all[:,1])[0,1]
    
    #draw correlation map
    main = go.Scatter(x = np.squeeze(descriptor_all), 
              y = segE_all[:,1], 
              marker={'color':'red', 'symbol':'circle','size':3}, 
              mode='markers'
             )

    data = [main]

    layout = go.Layout(autosize = False, 
               height = 800,
               width = 800, 
               xaxis={'title':descriptor_Name, 'zeroline':False},
               yaxis = {'title':'Segregation Energy(eV)', 'zeroline':False},
               annotations = [
                       dict(
                       x = 0,
                       y = 1,
                       xref = 'paper',
                       yref = 'paper',
                       text = '$\\text{R= } %4.2f$'%(R),
                       showarrow = False,
                       font = dict(size = 12)
                       )
                   ]
              )
    plot = py.iplot(go.Figure(data = data, layout = layout))

In [23]:
draw_correlation_segE_descriptor(mat, 'stress', 0)

In [1]:
def correlation_coefficients(mat, absvalue = False, export = False):
    descriptor_keys = ['$pe$','$pos_{x}$','$pos_{y}$','$pos_{z}$','$CNA$','$Centro_{FNN}$',
                   '$Centro_{SNN}$','$Coord$','$f_{x}$','$f_{y}$','$f_{z}$','$\sigma_{x}$',
                   '$\sigma_{y}$','$\sigma_{z}$','$\sigma_{xy}$','$\sigma_{xz}$','$\sigma_{yz}$',
                   '$Vor_{vol}$','$Vor_{faces}$','$\sigma_{H}$','$f_{mag}$','$E_{seg}$']
    #descriptor = {key:None for key in descriptor_keys }
    #descriptor_name = ['pe','pos','cna','centro_fnn','centro_snn','coord','f','stress','voronoi']

    for i in range(30):
        segE = mat['A']['Eseg'][0,i]
        #check whether this is a valid data?
        n1 = segE[:,0] != 0 
        segE = np.squeeze(segE[n1,:])
        atom_ID = segE[:,0].astype(int) - 1

        descriptor = mat['A']['peratom'][0,i][0,0]
        descriptor_temp = np.concatenate([descriptor['pos'],descriptor['pe'],descriptor['cna'],descriptor['centro_fnn'],
                                     descriptor['centro_snn'],descriptor['coord'],descriptor['f'],descriptor['stress'],
                                     descriptor['voronoi']], axis = 1)
        if i == 0:
            descriptor_all = descriptor_temp[atom_ID]
            segE_all = segE
        else:
            descriptor_temp = descriptor_temp[atom_ID]
            descriptor_all = np.concatenate([descriptor_all, descriptor_temp], axis = 0)
            segE_all = np.concatenate([segE_all, segE])

    descriptor_all[:,2] = abs(descriptor_all[:,2]-min(descriptor_all[:,2])-20)
    sigma_H = np.sum(descriptor_all[:,11:13], axis = 1)/3
    f_mag = np.linalg.norm(descriptor_all[:,8:11], axis = 1, ord = 2)

    feature = np.concatenate([descriptor_all, sigma_H[:,np.newaxis], f_mag[:,np.newaxis], segE_all[:,1][:,np.newaxis]], axis = 1)
    #calculate correlation coefficient
    R = np.corrcoef(feature, rowvar = False)
    
    #Optimal Leaf Algorithm for improving the order of the correlations
    import scipy.spatial.distance
    import scipy.cluster.hierarchy
    D = scipy.spatial.distance.pdist(R)
    tree = scipy.cluster.hierarchy.linkage(D, 'average')
    leafOrder = scipy.cluster.hierarchy.dendrogram(tree, no_plot = True)
    leafOrder = leafOrder['leaves']
    R = R[leafOrder,:]
    R = R[:,leafOrder]
    descriptor_keys = np.array(descriptor_keys)[leafOrder]
    if absvalue == True:
        R = abs(R)
    trace = go.Heatmap(z=R,
                   x=descriptor_keys,
                   y=descriptor_keys,
                   colorscale = 'Portland')
    layout = dict(height = 1000, width = 1000)
    figure=dict(data = [trace],layout = layout)
    py.iplot(figure)

In [5]:
correlation_coefficients(mat, absvalue = True)